In [1]:
import numpy as np
import pandas as pd

def calc_Rs(R1, R2, A):
    midR = (R1+R2)/2
    Ro = (A/np.pi+4*midR**2)/4/midR
    Ri = 2*midR - Ro   
    return Ri, midR, Ro

# Second Model 

## Geometric definition

In [2]:
# Rings radius
P = 18  # cm
F = P/np.cos(np.pi/6)
AF = F/2 * P * 6

AIR = 19*AF
AR1 = 18*AF
AR2 = 24*AF
AR3 = 24*AF

RIR = np.sqrt(1/np.pi*AIR)
print('RIR: ', np.round(RIR, 2))
RR1 = np.sqrt(1/np.pi*(AIR+AR1))
print('RR1: ', np.round(RR1, 2))
RR2 = np.sqrt(1/np.pi*(AIR+AR1+AR2))
print('RR2: ', np.round(RR2, 2))
RR3 = np.sqrt(1/np.pi*(AIR+AR1+AR2+AR3))
print('RR3: ', np.round(RR3, 2))

RIR:  82.39
RR1:  114.97
RR2:  147.62
RR3:  174.26


In [3]:
# Coolant channel radii
Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac

# R1 coolant
R1Ri, midR1, R1Ro = calc_Rs(RIR, RR1, AcR1)
print(np.round(calc_Rs(RIR, RR1, AcR1), 2))

# R2 coolant
R2Ri, midR2, R2Ro = calc_Rs(RR1, RR2, AcR2)
print(np.round(calc_Rs(RR1, RR2, AcR2), 2))

# R3 coolant
R3Ri, midR3, R3Ro = calc_Rs(RR2, RR3, AcR3)
print(np.round(calc_Rs(RR2, RR3, AcR3), 2))

[ 95.12  98.68 102.24]
[127.73 131.3  134.87]
[158.03 160.94 163.85]


In [51]:
# RPV
t = 340.8-327.5  #cm
R = 297.3 + t
print(R)

k = 40.0 # W/m/K

310.6


## Power density

In [10]:
# power density averaged in the ring (divided into layers)
file = pd.read_csv('power.csv')

# W/cm3
R1Fs = np.zeros(10)
for i in range(1, 7):
    fi = np.array(file['F'+str(i)].tolist())
    R1Fs += fi/6/1e6

R2Fs = np.zeros(10)
for i in range(7, 15):
    fi = np.array(file['F'+str(i)].tolist())
    R2Fs += fi/8/1e6

R3Fs = np.zeros(10)
for i in range(15, 23):
    fi = np.array(file['F'+str(i)].tolist())
    R3Fs += fi/8/1e6

QR1s = R1Fs*AR1/(AR1-AcR1)
QR2s = R2Fs*AR2/(AR2-AcR2)
QR3s = R3Fs*AR3/(AR3-AcR3)
print('Ring one power density: ', QR1s)
print('Ring two power density: ', QR2s)
print('Ring three power density: ', QR3s)

Ring one power density:  [ 4.9573133   5.68414705  6.43529391  7.22205732  8.24619338  9.83422275
 11.11919146 12.03093274 11.61355785  8.95553876]
Ring two power density:  [3.88017895 4.55342791 5.33080732 5.97094567 6.74256671 8.11513723
 9.00992332 9.59055856 8.97985186 6.61380325]
Ring three power density:  [ 4.21320445  4.99842164  5.82826616  6.5459825   7.44364777  9.06654726
 10.14528115 10.83100657 10.10481264  7.13125642]


## Coolant properties

In [19]:
# helium velocity
m = 157.1  # kg/s
At = AcR1 + AcR2 + AcR3
rho = 5.7030e-6  # kg/cm3
v = m/rho/At
print(v)  # cm/s

1701.709941080526


In [56]:
# simple - film
rho = 5.7030  # [kg/m3] 6.4 MPa 259 C
cp = 5.188  # [J/g/K] 6.4 MPa 259 C [2]
cp *= 1e3  # [J/kg/K]
mu = 2.9744e-5  # Pa.s 6.4 MPa 259 C
v = 1701.71  # cm/s
v /= 100  # m/s

ri = R3Ri/100 # m
ro = R3Ro/100
Ah = np.pi*(ro**2-ri**2)
Ph = 2*np.pi*(ri+ro)
Dh = 4*Ah/Ph
Re = rho * v * Dh / mu
print('Re: ', Re)

k = 0.41 # W/m/K
Pr = mu * cp/k
print('Pr: ', Pr)

Nu = 0.023*(Re**0.8)*(Pr**0.4)
print('Nu: ', Nu)
h = Nu/Dh * k  # W/m^2/K
print('h: ', h)

h /= 1e4  # W/cm^2/K

ri *= 100  # cm
ro *= 100  # cm
dr = 0.01  # cm

kgci = h * (ri + dr) * np.log(1 + dr/ri)
kgco = h * (ro + dr) * np.log(1 + dr/ro)
print('ki: ', kgci)  # W/cm/C
print('ko: ', kgco)  # W/cm/C

Re:  379858.61335534276
Pr:  0.37637041951219513
Nu:  452.5552548816472
h:  1593.7604317472117
ki:  0.0015938108557548198
ko:  0.0015938090644267088


In [57]:
k1 = 0.001539
k2 = 0.001530
k3 = 0.001594

print((k1+k2+k3)/3)

0.0015543333333333331


## Unit cell model definition

In [11]:
# unitcell - R1 w/layers - power density
Af = 210*np.pi*0.6225**2
Qf1 = R1Fs*AR1/18/Af
print(Qf1)
Qf2 = R2Fs*AR1/18/Af
print(Qf2)
Qf3 = R3Fs*AR1/18/Af
print(Qf3)

[17.00776986 19.50142327 22.07849113 24.77775387 28.29140517 33.73968663
 38.14821414 41.27625648 39.84430821 30.7250587 ]
[13.31228967 15.62210198 18.28916966 20.48538465 23.13269628 27.84177195
 30.91164368 32.90371275 30.80847316 22.69092889]
[14.45484829 17.14880616 19.99587347 22.45824645 25.5379962  31.10591157
 34.80687956 37.15949667 34.66803838 24.46623012]


In [26]:
# unitcell - film
rho = 5.7030  # [kg/m3] 6.4 MPa 259 C
cp = 5.188  # [J/g/K] 6.4 MPa 259 C [2]
cp *= 1e3  # [J/kg/K]
mu = 2.9744e-5  # Pa.s 6.4 MPa 259 C
v = 1701.71  # cm/s
v /= 100  # m/s
rc = 1.588/2  # cm
D = 2 * rc / 100  # m
Re = rho * v * D / mu
print('Re: ', Re)

k = 0.41 # W/m/K
Pr = mu * cp/k
print('Pr: ', Pr)

Nu = 0.023*(Re**0.8)*(Pr**0.4)
print('Nu: ', Nu)
h = Nu/D * k  # W/m^2/K
print('h: ', h)

rc = 1.588/2  # cm
dr = 0.01  # cm

h /= 1e4  # W/cm^2/K
kgc = h * (rc + dr) * np.log(1 + dr/rc)
print('kgc: ', kgc)  # W/cm/C

Re:  51813.156207773005
Pr:  0.37637041951219513
Nu:  91.94456614236985
h:  2373.8836346581634
kgc:  0.002388770158561358
